###DATA UPLOAD

testing on colab

In [3]:
!git clone https://github.com/nelson-nsc/ucl-nlp.git

Cloning into 'ucl-nlp'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 52 (delta 11), reused 33 (delta 3), pack-reused 0
Unpacking objects: 100% (52/52), 1.30 MiB | 3.29 MiB/s, done.
Filtering content: 100% (11/11), 158.87 MiB | 19.36 MiB/s, done.


In [6]:
import pandas as pd
train_df = pd.read_csv('/content/ucl-nlp/data/conala-hq-gpt2/hq_train_gpt2.csv')
train_df.head()

,0
0,convert a list to a dictionary in python = b =...
1,python - sort a list of nested lists = l.sort(...
2,how to get the size of a string in python? = p...
3,how to get the fft of a numpy array to work? =...
4,calculating difference between two rows in pyt...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###INSTALL DEPENDENCIES AND IMPORT LIBRARIES

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 836.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
torch.cuda.empty_cache()

PREPARE DATA


In [ ]:
# ## regular data
# df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/conala_alltrain.csv')

# # put in form we desire
# df = df['intent'] + ' = '  + df['snippet']

# # get number of sampels we wish to use
# df = df.iloc[0:20000]

In [ ]:
## high quality data
train_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/hq_train_gpt2.csv')
test_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/hq_test_gpt2.csv')
val_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/hq_val_gpt2.csv')

###TOKENIZER AND MODEL

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',  bos_token='<|startoftext|>',  eos_token='<|endoftext|>', pad_token='<|pad|>')
#model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/fineTunedGPT2_hq').cuda() # continue training
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

###PREPARE DATA PART 2

In [ ]:
max_length = max([len(tokenizer.encode(conala)) for conala in train_df['0']])

In [ ]:
class coNaLaDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for i in data['0']:
            encodings_dict = tokenizer('<|startoftext|>' + i + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
train_dataset = coNaLaDataset(train_df, tokenizer, max_length=max_length)
val_dataset = coNaLaDataset(val_df, tokenizer, max_length=max_length)

In [ ]:
my_data_collator = lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/GPT2-FineTuned',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    per_device_train_batch_size=2, 
    per_device_eval_batch_size=2,
    num_train_epochs=3, 
    gradient_accumulation_steps = 5,
    warmup_steps=10, 
    weight_decay=0.05, 
    report_to = 'none',
    save_total_limit = 2,
    load_best_model_at_end = True,
    )


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator = my_data_collator
)

trainer.train()

trainer.save_model('/content/drive/MyDrive/fineTunedGPT2_hq')
trainer.save_state()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.125300,0.129454
1,0.108100,0.118961
2,0.104300,0.115103


###GENERATED DESCRIPTION

In [ ]:
# define the tokenizer and load the trained model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',  bos_token='<|startoftext|>',  eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/fineTunedGPT2_hq').cuda() # continue training
model.resize_token_embeddings(len(tokenizer))

In [ ]:
generated = tokenizer("<|startoftext|>  get first element of python list = ", return_tensors="pt").input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=50, top_p=0.95, temperature=0.1, num_return_sequences=5,
                                pad_token_id=tokenizer.encode('<|pad|>')[0])

In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0:   get first element of python list =   a[0]
1:   get first element of python list =   a[0]
2:   get first element of python list =   a[0]
3:   get first element of python list =   a[0]
4:   get first element of python list =   a[0]
